In [28]:
import  numpy as np
import pandas as pd
import json
from google.colab import drive
drive.mount('/content/drive')

#setting tpu config
# import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
# !pip install tensorflow-text
!pip install -U "tensorflow==2.9.*"
!pip install -q -U "tensorflow-text==2.9.*"
import tensorflow as tf
import tensorflow_text as tf_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
df=pd.DataFrame(columns=["Key","Value"])
df.columns

Index(['Key', 'Value'], dtype='object')

In [31]:
import json
from sklearn.preprocessing import LabelBinarizer

with open('/content/drive/MyDrive/train.json', 'r') as f:
    train = json.load(f)

y = list(train.values())
x = list(train.keys())

In [32]:
df["Key"]=pd.Series(x[:4152659])
df["Value"]=pd.Series(y[:4152659])

In [33]:
df.tail(5)

,Key,Value
4152654,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152655,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152656,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152657,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152658,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal


In [34]:
import tensorflow as tf
import tensorflow_hub as hub

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [35]:
df['Value'].value_counts()

normal      4082967
abnormal      69692
Name: Value, dtype: int64

In [36]:
df_normal=df[df["Value"]=="normal"]
df_abnormal=df[df["Value"]=="abnormal"]

In [37]:
df_normal.shape,df_abnormal.shape

((4082967, 2), (69692, 2))

In [38]:
df_abnormal_upscaled=df_abnormal.sample(df_normal.shape[0],replace=True)

In [39]:
df_abnormal_upscaled.shape

(4082967, 2)

In [40]:
df_balanced = pd.concat([df_normal, df_abnormal_upscaled])
df_balanced.shape

(8165934, 2)

In [41]:
df_balanced['Value'].value_counts()

normal      4082967
abnormal    4082967
Name: Value, dtype: int64

In [42]:
df_balanced['Category']=df_balanced['Value'].apply(lambda x: 1 if x=='normal' else 0)
df_balanced.sample(5)

,Key,Value,Category
1800670,1121252363 2005.07.13 R34-M0-ND-C:J17-U01 200...,normal,1
2996424,1125222209 2005.08.28 R42-M1-N5-C:J14-U01 200...,normal,1
574241,1118814358 2005.06.14 R02-M1-N0-C:J12-U11 200...,normal,1
4120720,1118540572 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal,0
4128116,1118541665 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal,0


In [43]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_balanced['Key'],df_balanced['Category'], stratify=df_balanced['Category'])

In [44]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']



In [45]:
p=get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)
print(p)

tf.Tensor(
[[-0.84351724 -0.51327276 -0.88845724 ... -0.74748844 -0.7531474
   0.91964495]
 [-0.87208337 -0.5054394  -0.94446665 ... -0.85847497 -0.71745324
   0.8808295 ]], shape=(2, 768), dtype=float32)


In [46]:
#BUILDING MODEL
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [47]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [48]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
history=model.fit(x_train, y_train,validation_split=0.3, epochs=10)

Epoch 1/10
   141/133973 [..............................] - ETA: 13:27:14 - loss: 0.5021 - accuracy: 0.8327 - precision: 0.8551 - recall: 0.7864